<a href="https://colab.research.google.com/github/pevening/study_code/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D%EC%8A%A4%ED%84%B0%EB%94%94_2%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0.데이터 파악

In [1]:
# 대리점 매출이력과 고객정보 데이터
# 입력 실수 나 데이터 누락등을 포함하는 '더러운 데이터' 처리법

uriage.csv -> 매출 이력, 19/01~19/07

kokyaku_daicho.xlsx -> 대리점에서 관리하는 고객정보

##1.데이터 읽기

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# 매출 이력
uriage_data = pd.read_csv('/content/gdrive/MyDrive/데이터분석 스터디/2장/uriage.csv')
uriage_data

# 상품명에 공백 존재, 대소문자 섞여있음
# 가격에 결측치 확인

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현
...,...,...,...,...
2994,2019-02-15 2:56,상품Y,2500.0,김정민
2995,2019-06-22 4:03,상품M,1300.0,김재원
2996,2019-03-29 11:14,상품Q,NaN,김지율
2997,2019-07-14 12:56,상품H,NaN,김승주


In [5]:
#고객정보
kokyaku_data = pd.read_excel('/content/gdrive/MyDrive/데이터분석 스터디/2장/kokyaku_daicho.xlsx')
kokyaku_data

#고객이름 공백 존재
#등록일 형식 각기 다름 확인

,고객이름,지역,등록일
0,김 현성,H시,2018-01-04 00:00:00
1,김 도윤,E시,42782
2,김 지한,A시,2018-01-07 00:00:00
3,김 하윤,F시,42872
4,김 시온,E시,43127
...,...,...,...
195,김 재희,G시,2017-06-20 00:00:00
196,김 도영,E시,2018-06-20 00:00:00
197,김 이안,F시,2017-04-29 00:00:00
198,김 시현,H시,2019-04-19 00:00:00


##2.데이터 오류 살펴보기

In [6]:
uriage_data['item_name'].head()

# 띄어쓰기 등 형식 오류 확인

0      상품A
1    상 품 S
2    상 품 a
3      상품Z
4      상품a
Name: item_name, dtype: object

In [7]:
uriage_data['item_price'].head()

# 결측치 등 오류 확인

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

## 3.데이터에 오류가 있는 상태로 집계해보자

In [8]:
uriage_data['purchase_date'] = pd.to_datetime(uriage_data['purchase_date'])
uriage_data['purchase_month'] = uriage_data['purchase_date'].dt.strftime('%Y%m')
# -> 날짜를 년월 형태로 변환. 집계 단위에 맞춰

res = uriage_data.pivot_table(index = 'purchase_month',columns = 'item_name',aggfunc='size',fill_value=0)
res

# 대문자와 소문자, 띄어쓰기 구분 되지 못함. 원래 같은 상품인데 다르게 집계된 것 확인
# 원래 26개(A~Z) 이지만 칼럼이 99개인것 확인 가능

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,1,1,0,0
201903,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


In [9]:
res = uriage_data.pivot_table(index = 'purchase_month',columns = 'item_name'
,values='item_price',aggfunc='sum',fill_value=0)
res

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1400,0,0,0,0,0,0,0,0,0,...,1100,1200,1500,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,2400,0,0,0,...,0,0,0,0,0,1900,2000,2200,0,0
201903,0,500,1300,1600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,2300,0,1700,...,0,0,0,0,0,1900,0,0,0,0
201905,0,0,0,0,1900,0,0,0,0,0,...,0,1200,0,0,0,0,0,0,0,2500
201906,0,0,0,0,0,2300,0,0,0,0,...,0,0,0,1600,0,0,0,0,2400,0
201907,0,0,0,0,0,0,0,0,0,0,...,0,0,1500,0,1800,0,0,0,0,0


##4.상품명 오류를 수정하자

In [10]:
# 현황 파악
print(len(pd.unique(uriage_data.item_name)))

# 26개여야 하는것이 99개임을 확인

99


In [11]:
uriage_data['item_name'] = uriage_data['item_name'].str.upper()
#uriage_data
# 대문자로 변환

uriage_data['item_name'] = uriage_data['item_name'].str.replace(' ','')
# uriage_data
# 공백 없도록 조정

uriage_data.sort_values(by=['item_name'],ascending=True)
# item_name 순으로 데이터 정렬

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1748,2019-05-19 20:22:00,상품A,100.0,김시훈,201905
223,2019-06-25 08:13:00,상품A,100.0,김유진,201906
1742,2019-06-13 16:03:00,상품A,100.0,김건희,201906
1738,2019-02-10 00:28:00,상품A,100.0,김하랑,201902
...,...,...,...,...,...
2880,2019-04-22 00:36:00,상품Y,NaN,김동욱,201904
2881,2019-04-30 14:21:00,상품Y,NaN,김하준,201904
1525,2019-01-24 10:27:00,상품Y,2500.0,김범준,201901
1361,2019-05-28 13:45:00,상품Y,2500.0,김수현,201905


In [12]:
# 잘 수정되었는지 검증단계

print(pd.unique(uriage_data['item_name']))
# 중복을 제거하고 확인

print('')
print(len(pd.unique(uriage_data['item_name'])))

['상품A' '상품S' '상품Z' '상품V' '상품O' '상품U' '상품L' '상품C' '상품I' '상품R' '상품X' '상품G'
 '상품P' '상품Q' '상품Y' '상품N' '상품W' '상품E' '상품K' '상품B' '상품F' '상품D' '상품M' '상품H'
 '상품T' '상품J']

26


## 5.금액의 결측치를 수정하자

In [17]:
#결측치 확인

uriage_data.isnull().any(axis=0)
# any(axis=0) -> 열에 결측치 있는지 없는지 확인 / 금액에만 결측치 있는것 확인
# uriage_data

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

In [ ]:
# 결측치 확인 함수

# def check_missing_col(dataframe):
#     missing_col = []
#     counted_missing_col = 0
#     for i,col in enumerate(dataframe.columns):
#         missing_values = sum(dataframe[col].isna())
#         is_missing = True if missing_values >= 1 else False
#         if is_missing:
#             counted_missing_col +=1
#             print('결측치가 있는 칼럼은: %s입니다.'%col)
#             print('해당 칼럼에 총 %s개의 결측치가 존재합니다. '%missing_values)
#             missing_col.append([col, dataframe[col].dtype])
#     if counted_missing_col == 0:
#         print('결측치가 존재하지 않습니다.')
#     return missing_col

In [ ]:
# check_missing_col(uriage_data)

In [24]:
#밑에 코드 한줄씩 뜯어보자 !

# uriage_data['item_price'].isnull()
# uriage_data['item_price'].isnull().value_counts()
# -> 결측치 수 387개
# flg_is_null = uriage_data['item_price'].isnull()
# uriage_data.loc[flg_is_null, 'item_name']
# -> 가격이 결측치인 상품의 item name 출력, 387개인 것 볼 수 있음

# uriage_data.loc[flg_is_null, 'item_name'].unique()
# 리스트에 들어간것, unique 로 중복제거

# flg_is_null = uriage_data['item_price'].isnull()
# for trg in list(uriage_data.loc[flg_is_null, 'item_name'].unique()):
  # price = uriage_data.loc[(~flg_is_null) & (uriage_data['item_name']==trg), 'item_price'].max()
# price
# ~fig_is_null 을 통해 결측치 없는 행이면서 and 아까 trg 에 들어간 같은상품 가격의 최고가를 결측치 자리에 채우기 위해 가져옴

# uriage_data['item_price'].loc[(flg_is_null)&(uriage_data['item_name']==trg)]=price
#price로 결측치를 채워준다

array(['상품S', '상품A', '상품P', '상품N', '상품W', '상품R', '상품I', '상품L', '상품F',
       '상품O', '상품B', '상품C', '상품V', '상품Q', '상품U', '상품K', '상품T', '상품X',
       '상품E', '상품M', '상품G', '상품J', '상품D', '상품H', '상품Y'], dtype=object)

In [25]:
#같은상품의 단가로 결측치 채우기

flg_is_null = uriage_data['item_price'].isnull()
for trg in list(uriage_data.loc[flg_is_null, 'item_name'].unique()):
  price = uriage_data.loc[(~flg_is_null) & (uriage_data['item_name']==trg), 'item_price'].max()
  uriage_data['item_price'].loc[(flg_is_null)&(uriage_data['item_name']==trg)]=price
uriage_data.head()

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상품S,1900.0,김우찬,201907
2,2019-05-11 19:42:00,상품A,100.0,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품A,100.0,김강현,201904


In [26]:
#검증 진행
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price        False
customer_name     False
purchase_month    False
dtype: bool

In [27]:
# 금액이 정상 수정되었는지 반복문으로 확인
# 그냥 최고가와 최저가가 같게 올바르게 설정되었다는것 보여줌

for trg in list(uriage_data['item_name'].sort_values().unique()):
  print(trg + "의 최고가:" + str(uriage_data.loc[uriage_data['item_name']==trg]['item_price'].max())
          + "의 최저가:" + str(uriage_data.loc[uriage_data['item_name']==trg]['item_price'].min(skipna=False))) 
  
#skipna -> NaN의 무시여부. 

상품A의 최고가:100.0의 최저가:100.0
상품B의 최고가:200.0의 최저가:200.0
상품C의 최고가:300.0의 최저가:300.0
상품D의 최고가:400.0의 최저가:400.0
상품E의 최고가:500.0의 최저가:500.0
상품F의 최고가:600.0의 최저가:600.0
상품G의 최고가:700.0의 최저가:700.0
상품H의 최고가:800.0의 최저가:800.0
상품I의 최고가:900.0의 최저가:900.0
상품J의 최고가:1000.0의 최저가:1000.0
상품K의 최고가:1100.0의 최저가:1100.0
상품L의 최고가:1200.0의 최저가:1200.0
상품M의 최고가:1300.0의 최저가:1300.0
상품N의 최고가:1400.0의 최저가:1400.0
상품O의 최고가:1500.0의 최저가:1500.0
상품P의 최고가:1600.0의 최저가:1600.0
상품Q의 최고가:1700.0의 최저가:1700.0
상품R의 최고가:1800.0의 최저가:1800.0
상품S의 최고가:1900.0의 최저가:1900.0
상품T의 최고가:2000.0의 최저가:2000.0
상품U의 최고가:2100.0의 최저가:2100.0
상품V의 최고가:2200.0의 최저가:2200.0
상품W의 최고가:2300.0의 최저가:2300.0
상품X의 최고가:2400.0의 최저가:2400.0
상품Y의 최고가:2500.0의 최저가:2500.0
상품Z의 최고가:2600.0의 최저가:2600.0


##6.고객 이름의 오류를 수정하자

In [28]:
kokyaku_data['고객이름'].head()

0    김 현성
1    김 도윤
2    김 지한
3    김 하윤
4    김 시온
Name: 고객이름, dtype: object

In [29]:
uriage_data['customer_name'].head()

0    김가온
1    김우찬
2    김유찬
3    김재현
4    김강현
Name: customer_name, dtype: object

In [30]:
kokyaku_data['고객이름'] = kokyaku_data['고객이름'].str.replace("  ","")
kokyaku_data['고객이름'] = kokyaku_data['고객이름'].str.replace(" ","")
kokyaku_data['고객이름'].head()

0    김현성
1    김도윤
2    김지한
3    김하윤
4    김시온
Name: 고객이름, dtype: object

##7.날짜 오류를 수정하자

In [31]:
kokyaku_data
# 등록일 보면 날짜 형식이 아닌 숫자 등 서식이 다른 데이터 섞여있음

,고객이름,지역,등록일
0,김현성,H시,2018-01-04 00:00:00
1,김도윤,E시,42782
2,김지한,A시,2018-01-07 00:00:00
3,김하윤,F시,42872
4,김시온,E시,43127
...,...,...,...
195,김재희,G시,2017-06-20 00:00:00
196,김도영,E시,2018-06-20 00:00:00
197,김이안,F시,2017-04-29 00:00:00
198,김시현,H시,2019-04-19 00:00:00


In [32]:
flg_is_serial = kokyaku_data['등록일'].astype('str').str.isdigit()
flg_is_serial.sum()
# 숫자로 읽히는 데이터 (ex> 42782) 의 개수 확인
# str.isdigit -> 문자열의 모든 문자가 숫자일 때 True

22

In [33]:
fromSerial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial, '등록일'].astype('float'),unit='D') + pd.to_datetime('1900/01/01')
fromSerial

# to_timedelta -> 숫자를 날짜로, unit='D' -> 시간 간격 단위를 day
# 결국 숫자 형태의 40000일 형식으로 바꾸고 거기에 1900년 1월 1일을 더해주는 꼴

1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 등록일, dtype: datetime64[ns]

In [34]:
fromString = pd.to_datetime(kokyaku_data.loc[~flg_is_serial,'등록일'])
fromString

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 등록일, Length: 178, dtype: datetime64[ns]

In [35]:
# 두개 위아래로 합쳐서 다시 등록일 열 만듬

kokyaku_data['등록일'] = pd.concat([fromSerial,fromString])
kokyaku_data

,고객이름,지역,등록일
0,김현성,H시,2018-01-04
1,김도윤,E시,2017-02-18
2,김지한,A시,2018-01-07
3,김하윤,F시,2017-05-19
4,김시온,E시,2018-01-29
...,...,...,...
195,김재희,G시,2017-06-20
196,김도영,E시,2018-06-20
197,김이안,F시,2017-04-29
198,김시현,H시,2019-04-19


In [36]:
#등록연월 별 고객수 그룹화하여 출력

kokyaku_data['등록연월'] = kokyaku_data['등록일'].dt.strftime('%Y%m')
rslt = kokyaku_data.groupby('등록연월').count()['고객이름']
print(rslt)

print(len(kokyaku_data))

등록연월
201701    15
201702    11
201703    14
201704    15
201705    13
201706    14
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 고객이름, dtype: int64
200


In [37]:
# 등록일에 날짜로 변환되지 않고 숫자로 남아있는 데이터 있는지 확인

flg_is_serial = kokyaku_data['등록일'].astype('str').str.isdigit()
flg_is_serial.sum()

0

##8.고객 이름을 키로 두 개의 데이터를 결합(조인)하자

In [38]:
uriage_data.head()

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상품S,1900.0,김우찬,201907
2,2019-05-11 19:42:00,상품A,100.0,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품A,100.0,김강현,201904


In [39]:
kokyaku_data.head()

,고객이름,지역,등록일,등록연월
0,김현성,H시,2018-01-04,201801
1,김도윤,E시,2017-02-18,201702
2,김지한,A시,2018-01-07,201801
3,김하윤,F시,2017-05-19,201705
4,김시온,E시,2018-01-29,201801


In [ ]:
# 두 개의 데이터 조인해야함. 그런데 공통키로 보이는 칼럼이름이 존재 x
# 따라서 서로 다른 열을 지정해서 결합해야함


In [40]:
join_data = pd.merge(uriage_data,kokyaku_data
                     ,left_on='customer_name', right_on='고객이름'
                     ,how = 'left')
join_data

,purchase_date,item_name,item_price,customer_name,purchase_month,고객이름,지역,등록일,등록연월
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906,김가온,C시,2017-01-26,201701
1,2019-07-13 13:05:00,상품S,1900.0,김우찬,201907,김우찬,C시,2018-04-07,201804
2,2019-05-11 19:42:00,상품A,100.0,김유찬,201905,김유찬,A시,2018-06-19,201806
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902,김재현,D시,2018-07-22,201807
4,2019-04-22 03:09:00,상품A,100.0,김강현,201904,김강현,D시,2017-06-07,201706
...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,상품Y,2500.0,김정민,201902,김정민,B시,2017-07-01,201707
2995,2019-06-22 04:03:00,상품M,1300.0,김재원,201906,김재원,E시,2018-03-31,201803
2996,2019-03-29 11:14:00,상품Q,1700.0,김지율,201903,김지율,B시,2017-03-15,201703
2997,2019-07-14 12:56:00,상품H,800.0,김승주,201907,김승주,E시,2018-07-15,201807


In [41]:
join_data = join_data.drop('customer_name',axis=1)
join_data

,purchase_date,item_name,item_price,purchase_month,고객이름,지역,등록일,등록연월
0,2019-06-13 18:02:00,상품A,100.0,201906,김가온,C시,2017-01-26,201701
1,2019-07-13 13:05:00,상품S,1900.0,201907,김우찬,C시,2018-04-07,201804
2,2019-05-11 19:42:00,상품A,100.0,201905,김유찬,A시,2018-06-19,201806
3,2019-02-12 23:40:00,상품Z,2600.0,201902,김재현,D시,2018-07-22,201807
4,2019-04-22 03:09:00,상품A,100.0,201904,김강현,D시,2017-06-07,201706
...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,상품Y,2500.0,201902,김정민,B시,2017-07-01,201707
2995,2019-06-22 04:03:00,상품M,1300.0,201906,김재원,E시,2018-03-31,201803
2996,2019-03-29 11:14:00,상품Q,1700.0,201903,김지율,B시,2017-03-15,201703
2997,2019-07-14 12:56:00,상품H,800.0,201907,김승주,E시,2018-07-15,201807


##9.정제한 데이터를 덤프(출력)하자

In [42]:
#데이터 출력 전 칼럼 배치 수정

dump_data = join_data[['purchase_date','purchase_month','item_name','item_price','고객이름','지역','등록일']]
dump_data

,purchase_date,purchase_month,item_name,item_price,고객이름,지역,등록일
0,2019-06-13 18:02:00,201906,상품A,100.0,김가온,C시,2017-01-26
1,2019-07-13 13:05:00,201907,상품S,1900.0,김우찬,C시,2018-04-07
2,2019-05-11 19:42:00,201905,상품A,100.0,김유찬,A시,2018-06-19
3,2019-02-12 23:40:00,201902,상품Z,2600.0,김재현,D시,2018-07-22
4,2019-04-22 03:09:00,201904,상품A,100.0,김강현,D시,2017-06-07
...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,201902,상품Y,2500.0,김정민,B시,2017-07-01
2995,2019-06-22 04:03:00,201906,상품M,1300.0,김재원,E시,2018-03-31
2996,2019-03-29 11:14:00,201903,상품Q,1700.0,김지율,B시,2017-03-15
2997,2019-07-14 12:56:00,201907,상품H,800.0,김승주,E시,2018-07-15


In [43]:
dump_data.to_csv('dump_data.csv',index=False)

##10.데이터를 집계하자

In [44]:
import_data = pd.read_csv('dump_data.csv')
import_data

,purchase_date,purchase_month,item_name,item_price,고객이름,지역,등록일
0,2019-06-13 18:02:00,201906,상품A,100.0,김가온,C시,2017-01-26 00:00:00
1,2019-07-13 13:05:00,201907,상품S,1900.0,김우찬,C시,2018-04-07 00:00:00
2,2019-05-11 19:42:00,201905,상품A,100.0,김유찬,A시,2018-06-19 00:00:00
3,2019-02-12 23:40:00,201902,상품Z,2600.0,김재현,D시,2018-07-22 00:00:00
4,2019-04-22 03:09:00,201904,상품A,100.0,김강현,D시,2017-06-07 00:00:00
...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,201902,상품Y,2500.0,김정민,B시,2017-07-01 00:00:00
2995,2019-06-22 04:03:00,201906,상품M,1300.0,김재원,E시,2018-03-31 00:00:00
2996,2019-03-29 11:14:00,201903,상품Q,1700.0,김지율,B시,2017-03-15 00:00:00
2997,2019-07-14 12:56:00,201907,상품H,800.0,김승주,E시,2018-07-15 00:00:00


In [45]:
#purchase_month 를 세로축으로 하여 상품별 pivot table 새로 생성

byItem = import_data.pivot_table(index='purchase_month',columns='item_name',aggfunc='size',fill_value=0)
byItem

item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,...,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,18,13,19,17,18,15,11,16,18,17,...,17,21,20,17,7,22,13,14,10,0
201902,19,14,26,21,16,14,14,17,12,14,...,22,22,22,23,19,22,24,16,11,1
201903,17,21,20,17,9,27,14,18,12,16,...,23,16,20,12,23,18,16,21,16,0
201904,17,19,24,20,18,17,14,11,18,13,...,20,20,16,16,11,15,14,16,20,0
201905,24,14,16,14,19,18,23,15,16,11,...,13,22,18,16,16,9,21,16,20,0
201906,24,12,11,19,13,18,15,13,19,22,...,15,16,21,12,18,20,17,15,13,0
201907,20,20,17,17,12,17,19,19,19,23,...,15,19,23,21,13,28,16,18,12,0


In [46]:
#purchase_month 를 세로축으로 하여 매출 금액별 pivot table 새로 생성

byPrice = import_data.pivot_table(index='purchase_month',columns='item_name',
                                  values='item_price',aggfunc='sum',fill_value=0)
byPrice

item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,...,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1800,2600,5700,6800,9000,9000,7700,12800,16200,17000,...,28900,37800,38000,34000,14700,48400,29900,33600,25000,0
201902,1900,2800,7800,8400,8000,8400,9800,13600,10800,14000,...,37400,39600,41800,46000,39900,48400,55200,38400,27500,2600
201903,1700,4200,6000,6800,4500,16200,9800,14400,10800,16000,...,39100,28800,38000,24000,48300,39600,36800,50400,40000,0
201904,1700,3800,7200,8000,9000,10200,9800,8800,16200,13000,...,34000,36000,30400,32000,23100,33000,32200,38400,50000,0
201905,2400,2800,4800,5600,9500,10800,16100,12000,14400,11000,...,22100,39600,34200,32000,33600,19800,48300,38400,50000,0
201906,2400,2400,3300,7600,6500,10800,10500,10400,17100,22000,...,25500,28800,39900,24000,37800,44000,39100,36000,32500,0
201907,2000,4000,5100,6800,6000,10200,13300,15200,17100,23000,...,25500,34200,43700,42000,27300,61600,36800,43200,30000,0


In [47]:
#purchase_month 를 세로축으로 하여 고객별 pivot table 새로 생성

byCustomer = import_data.pivot_table(index='purchase_month',columns='고객이름',aggfunc='size',fill_value=0)
byCustomer

고객이름,김가온,김강민,김강현,김건우,김건희,김경민,김규민,김규현,김다온,김대현,...,김현수,김현승,김현우,김현준,김현진,김호준,정도형,정영훈,정우석,정준기
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1,2,1,2,5,3,1,1,1,5,...,3,2,4,2,1,1,4,4,4,3
201902,3,1,4,3,3,5,2,3,1,5,...,5,3,7,0,0,1,2,0,2,2
201903,4,0,3,1,1,2,2,1,1,5,...,3,2,1,2,1,2,1,2,3,4
201904,1,0,3,0,2,1,2,6,2,4,...,0,4,3,1,1,2,2,4,3,1
201905,0,0,2,2,1,4,6,3,3,1,...,2,2,1,4,3,0,0,0,1,2
201906,5,0,3,0,2,1,6,1,1,1,...,2,4,1,0,4,2,2,0,0,2
201907,3,1,3,2,2,1,4,1,4,3,...,2,1,0,4,0,2,6,1,2,3


In [48]:
#purchase_month 를 세로축으로 하여 지역별 pivot table 새로 생성

byRegion = import_data.pivot_table(index='purchase_month',columns='지역',aggfunc='size',fill_value=0)
byRegion

지역,A시,B시,C시,D시,E시,F시,G시,H시
purchase_month,,,,,,,,
201901,59,55,72,34,49,57,49,42
201902,71,46,65,48,61,52,43,63
201903,64,52,57,43,52,59,51,59
201904,64,48,54,45,48,58,40,52
201905,57,52,68,48,59,65,35,43
201906,53,47,61,30,51,51,58,58
201907,76,53,61,42,54,64,47,54


In [49]:
# 집계 기간에 구매이력이 없는 사용자 확인

away_data = pd.merge(uriage_data, kokyaku_data, left_on = 'customer_name', right_on='고객이름',
                     how = 'right')
away_data[away_data['purchase_date'].isnull()][['고객이름','등록일']]

,고객이름,등록일
2999,김서우,2019-04-23
